# Weekly Update of the COVID-19 Cases in Top Ten Zip Code Areas
### Prepared for NIH RADx-Up Project: Communities Fighting COVID
Center for Human Dynamics in the Mobile Age (HDMA) at San Diego State University <br>
Jessica Embury

## Import Statements

In [29]:
import pandas as pd
import arcpy
import webbrowser
import json
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer

## User Variables

In [2]:
###########################
###SET DATA THROUGH DATE###
###########################
most_recent_date = '02/06/2021'  # Change date daily using 'MM/DD/YYYY' format
date_range = '1/31/21 - 2/6/21'

##########################
###SET PATHS IN AND OUT###
##########################
#PATHS IN
#additional needed zip info - community name and population
path_in = '../covid_data/percents/covid_percents_{}.csv'.format(most_recent_date.replace('/',''))

#PATHS OUT
path_out = './data/radx_covid_upload.csv'
path_out2 = '../covid_data/radx_updates/radx_covid_update_{}.csv'.format(most_recent_date.replace('/',''))
#path out for dash updates, feature layer overwrite
path_out3 = '../covid_data/radx_updates/radx_covid_sites_update_{}.csv'.format(most_recent_date.replace('/',''))
path_out4 = '../covid_data/radx_updates/radx_covid_sites_upload.csv'

################################
###ARCGIS DETAILS FOR UPDATES###
################################
layer_id = '94f1c49aeb324fff91d35f8ab86c553f'
map_id = '6978702efc634f508003453e434417da'
mobilemap_id = '06bec9354a214a7ea0e781368eca5cd9'
dash = 'https://experience.arcgis.com/experience/906d9ccaa0894762ae7bfa8aa46d1809'

## Data Processing - Pandas

In [3]:
# create df using covid spreadsheet from most_recent_date
df = pd.read_csv(path_in)

df.head()

,Zipcode,Community,Latitude,Longitude,2018_population,Date,Confirmed Cases,Rate Per 100K,Daily Increased,Daily Change Rate*1000,7 Days Rolling Change*1000,7 Day Case Increase,percent_total,percent_daily
0,91901,Alpine,32.805710,-116.695537,17885,02/06/2021,1131,6323.73,8,7.12,3.98,31,0.50,0.70
1,91902,Bonita,32.671583,-117.015068,17375,02/06/2021,1204,6929.50,12,10.07,5.21,43,0.53,1.05
2,91910,Chula Vista,32.636413,-117.065653,82682,02/06/2021,8200,9917.52,42,5.15,4.78,269,3.64,3.67
3,91911,Chula Vista,32.607309,-117.050214,84626,02/06/2021,10016,11835.61,38,3.81,4.32,298,4.44,3.32
4,91913,Chula Vista,32.616267,-116.987495,49519,02/06/2021,4128,8336.19,21,5.11,4.33,123,1.83,1.84


In [4]:
# delete unwanted columns
del df['Daily Increased']
del df['Daily Change Rate*1000']
del df['percent_total']
del df['percent_daily']

df.head()

,Zipcode,Community,Latitude,Longitude,2018_population,Date,Confirmed Cases,Rate Per 100K,7 Days Rolling Change*1000,7 Day Case Increase
0,91901,Alpine,32.805710,-116.695537,17885,02/06/2021,1131,6323.73,3.98,31
1,91902,Bonita,32.671583,-117.015068,17375,02/06/2021,1204,6929.50,5.21,43
2,91910,Chula Vista,32.636413,-117.065653,82682,02/06/2021,8200,9917.52,4.78,269
3,91911,Chula Vista,32.607309,-117.050214,84626,02/06/2021,10016,11835.61,4.32,298
4,91913,Chula Vista,32.616267,-116.987495,49519,02/06/2021,4128,8336.19,4.33,123


In [5]:
# rename columns to match new schema
df = df.rename(columns = {'2018_population': 'Population 2018', 'Date': 'Date Range', 'Confirmed Cases': 'Accumulated Confirmed Cases', 
                         'Rate Per 100K': 'Accumulated Rate Per 100K', '7 Day Case Increase': '7-Day Confirmed Cases', 
                          '7 Days Rolling Change*1000': '7-Day Change Rate*1000'})

df['Date Range'] = date_range

df.head()

,Zipcode,Community,Latitude,Longitude,Population 2018,Date Range,Accumulated Confirmed Cases,Accumulated Rate Per 100K,7-Day Change Rate*1000,7-Day Confirmed Cases
0,91901,Alpine,32.805710,-116.695537,17885,1/31/21 - 2/6/21,1131,6323.73,3.98,31
1,91902,Bonita,32.671583,-117.015068,17375,1/31/21 - 2/6/21,1204,6929.50,5.21,43
2,91910,Chula Vista,32.636413,-117.065653,82682,1/31/21 - 2/6/21,8200,9917.52,4.78,269
3,91911,Chula Vista,32.607309,-117.050214,84626,1/31/21 - 2/6/21,10016,11835.61,4.32,298
4,91913,Chula Vista,32.616267,-116.987495,49519,1/31/21 - 2/6/21,4128,8336.19,4.33,123


In [6]:
# create columns, calculate values for daily average and weekly case rate per 100k residents
df['7-Day Case Average'] = round(df['7-Day Confirmed Cases']/7, 2)

df['7-Day Case Rate Per 100K'] = round(df['7-Day Confirmed Cases']/df['Population 2018']*100000, 2)

df.head()

,Zipcode,Community,Latitude,Longitude,Population 2018,Date Range,Accumulated Confirmed Cases,Accumulated Rate Per 100K,7-Day Change Rate*1000,7-Day Confirmed Cases,7-Day Case Average,7-Day Case Rate Per 100K
0,91901,Alpine,32.805710,-116.695537,17885,1/31/21 - 2/6/21,1131,6323.73,3.98,31,4.43,173.33
1,91902,Bonita,32.671583,-117.015068,17375,1/31/21 - 2/6/21,1204,6929.50,5.21,43,6.14,247.48
2,91910,Chula Vista,32.636413,-117.065653,82682,1/31/21 - 2/6/21,8200,9917.52,4.78,269,38.43,325.34
3,91911,Chula Vista,32.607309,-117.050214,84626,1/31/21 - 2/6/21,10016,11835.61,4.32,298,42.57,352.14
4,91913,Chula Vista,32.616267,-116.987495,49519,1/31/21 - 2/6/21,4128,8336.19,4.33,123,17.57,248.39


In [7]:
# save df to csv files
df.to_csv(path_out, index=False)
df.to_csv(path_out2, index=False)

## Data Processing - arcpy
Calculate number of County test sites in each zip code and in adjacent zip codes, add as columns to df

In [8]:
# csv path for gdb table
csv_in = path_out

# Get and set current project and geodatabase info
arcpy.env.overwriteOutput = True
aprx = arcpy.mp.ArcGISProject('CURRENT')
defaultGeoDb = arcpy.env.workspace
aprx.defaultGeodatabase = defaultGeoDb
currentMap = aprx.activeMap

# set names for table and layer
tempStr = 'radx_covid_update_{}.csv'.format(most_recent_date.replace('/',''))
newMapName= 'lyr_' + tempStr
newTableName = 'tbl_' + tempStr

# convert CSV data into table and add to default database - look in database to see it was added
arcpy.TableToTable_conversion(csv_in,defaultGeoDb,newTableName)

<Result 'C:\\Users\\jesse\\iCloudDrive\\HDMA\\covid_dashboards\\Default.gdb\\tbl_radx_covid_update_02062021'>

In [9]:
# join radx case data to zipcode polygon
covid_data_join = arcpy.management.AddJoin('zips_test_sites', 'ZIP', newTableName, 'Zipcode')

# output joined layer to gdb
arcpy.CopyFeatures_management(covid_data_join, 'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/{}'.format(newMapName).replace('.csv',''))

<Result 'C:\\Users\\jesse\\iCloudDrive\\HDMA\\covid_dashboards\\Default.gdb\\lyr_radx_covid_update_02062021'>

In [10]:
# remove join
arcpy.RemoveJoin_management('zips_test_sites')

<Result 'zips_test_sites'>

In [11]:
# spatial join to get number of test sites in each zip code
# Join_Count = number of tests sites in each zip code
zip_sites = arcpy.analysis.SpatialJoin('zips_test_sites', 'COVID19_Testing_Locations', 
                                       'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/sites_in_zips_{}'.format(most_recent_date.replace('/', '')))

In [ ]:
# get zip code neighbors - do not need to re-run each time
# zip_nbrs = arcpy.analysis.PolygonNeighbors('zips_test_sites', 'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/zip_neighbors', 'ZIP')

In [12]:
#join zip_nbrs to zip_sites to eventually get number of sites in neighboring zip codes for each zip code
nbrs_join = arcpy.management.AddJoin('sites_in_zips_{}'.format(most_recent_date.replace('/', '')), 'ZIP', 
                                     'zip_neighbors', 'nbr_ZIP')
# output joined layer to gdb
arcpy.CopyFeatures_management(nbrs_join, 'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/zip_nbr_join_{}'.format(most_recent_date.replace('/','')))

<Result 'C:\\Users\\jesse\\iCloudDrive\\HDMA\\covid_dashboards\\Default.gdb\\zip_nbr_join_02062021'>

In [13]:
# remove join
arcpy.RemoveJoin_management('sites_in_zips_{}'.format(most_recent_date.replace('/', '')))

<Result 'sites_in_zips_02062021'>

In [14]:
# dissolve zip_nbr_join to get sum of sites in neighboring zips
arcpy.management.Dissolve('C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/zip_nbr_join_{}'.format(most_recent_date.replace('/','')), 
                          'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/zip_nbr_diss_{}'.format(most_recent_date.replace('/','')), 
                          'zip_neighbors_src_ZIP', 'sites_in_zips_{}_Join_Count SUM'.format(most_recent_date.replace('/', '')))

<Result 'C:\\Users\\jesse\\iCloudDrive\\HDMA\\covid_dashboards\\Default.gdb\\zip_nbr_diss_02062021'>

In [15]:
# join zip_nbr_diss and sites_in_zips
sites_join_in_adj = arcpy.management.AddJoin('sites_in_zips_{}'.format(most_recent_date.replace('/', '')), 'ZIP', 
                                             'zip_nbr_diss_{}'.format(most_recent_date.replace('/', '')), 'zip_neighbors_src_ZIP')
# output joined layer to gdb
arcpy.CopyFeatures_management(sites_join_in_adj, 'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/zip_sites_in_and_adj_{}'.format(most_recent_date.replace('/','')))

<Result 'C:\\Users\\jesse\\iCloudDrive\\HDMA\\covid_dashboards\\Default.gdb\\zip_sites_in_and_adj_02062021'>

In [16]:
# remove join
arcpy.RemoveJoin_management('sites_in_zips_{}'.format(most_recent_date.replace('/', '')))

<Result 'sites_in_zips_02062021'>

In [17]:
# join zip_sites_in_and_adj and lyr_radx_covid_update
radx_sites_join = arcpy.management.AddJoin(newMapName, 'zips_test_sites_ZIP', 
                                             'zip_sites_in_and_adj_{}'.format(most_recent_date.replace('/', '')), 
                                             'sites_in_zips_{}_ZIP'.format(most_recent_date.replace('/','')))

# output joined layer to gdb
arcpy.CopyFeatures_management(radx_sites_join, 'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/radx_7day_update_{}'.format(most_recent_date.replace('/','')))

<Result 'C:\\Users\\jesse\\iCloudDrive\\HDMA\\covid_dashboards\\Default.gdb\\radx_7day_update_02062021'>

In [18]:
# remove join
arcpy.RemoveJoin_management(newMapName)

<Result 'lyr_radx_covid_update_02062021'>

In [19]:
# del unnecessary fields from radx_7day_update
arcpy.DeleteField_management('radx_7day_update_{}'.format(most_recent_date.replace('/','')), 
                             ['lyr_radx_covid_update_{}_zips_test_sites_ZIP'.format(most_recent_date.replace('/','')),
                             'lyr_radx_covid_update_{}_zips_test_sites_zip_text'.format(most_recent_date.replace('/','')), 
                             'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}_OB'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_OBJECTID'.format(most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_TARGET_FID'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_ZIP'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_zip_text'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_facilityid'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_name'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_fulladdr'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_municipalit'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_agency'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_agencytype'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_phone'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_agencyurl'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_operhours'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_comments'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_Instruction'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_status'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_CreationDat'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_EditDate'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_drive_throu'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_appt_only'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_referral_re'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_services_of'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_call_first'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_virtual_scr'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_health_dept'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_State'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_data_source'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_county'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_red_flag'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_vol_note'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_public_form'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_start_date'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_end_date'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_type_of_tes'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_test_proces'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_fine_print'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_bos_distric'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_DISPLAY_PUB'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_name_spanis'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_operhours_s'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_instructi_1'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_sites_in_zips_{}_operhours_n'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_zip_nbr_diss_{}_OBJECTID'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                             'zip_sites_in_and_adj_{}_zip_nbr_diss_{}_zip_neighbor'.format(most_recent_date.replace('/',''), most_recent_date.replace('/',''))
                             ])


<Result 'radx_7day_update_02062021'>

In [21]:
# rename fields
arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')), 
                            'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}_Zi'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'zipcode')

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}_Co'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'community', 'Community')

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}_La'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'lat', 'Latitude'
                           )

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}_Lo'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'lon', 'Longitude'
                           )

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}_Po'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'population2018', 'Population 2018'
                           )

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}_Da'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'date_range', 'Date Range'
                           )

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}_Ac'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'accum_cases', 'Accumulated Confirmed Cases'
                           )

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}__1'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'accum_rate_100k_res', 'Accumulated Rate Per 100K'
                           )

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}_F7'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'change_rate_7day', '7-Day Change Rate*1000'
                           )

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}__2'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'confirmed_cases_7day', '7-Day Confirmed Cases'
                           )

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}__3'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'case_avg_7day', '7-Day Case Average'
                           )

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'lyr_radx_covid_update_{}_tbl_radx_covid_update_{}__4'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'case_100kres_7day', '7-Day Case Rate Per 100K'
                           )

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'zip_sites_in_and_adj_{}_sites_in_zips_{}_Join_Count'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'sites_in_zip', 'County Test Sites in Zip'
                           )

arcpy.management.AlterField('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                            'zip_sites_in_and_adj_{}_zip_nbr_diss_{}_SUM_sites_in'.format(most_recent_date.replace('/',''), most_recent_date.replace('/','')),
                            'sites_adj_zips', 'County Test Sites in Adj. Zips'
                           )



<Result 'radx_7day_update_02062021'>

In [22]:
# save as csv file
arcpy.conversion.TableToTable('radx_7day_update_{}'.format(most_recent_date.replace('/','')),
                              '../covid_data/radx_updates',
                              'radx_covid_sites_update{}.csv'.format(most_recent_date.replace('/',''))
                             )

<Result '../covid_data/radx_updates\\radx_covid_sites_update02062021.csv'>

In [23]:
# format table for layer overwrite
df2 = pd.read_csv('../covid_data/radx_updates/radx_covid_sites_update{}.csv'.format(most_recent_date.replace('/','')))

del df2['OID_']
del df2 ['Shape_Length']
del df2['Shape_Area']

df2 = df2.rename(columns = {'zipcode': 'Zip Code',
                           'community': 'Community',
                           'lat': 'Latitude', 
                            'lon': 'Longitude',
                            'population2018': 'Population 2018',
                            'date_range': 'Date Range',
                            'accum_cases': 'Accumulated Confirmed Cases',
                            'accum_rate_100k_res': 'Accumulated Rate Per 100K',
                            'change_rate_7day': '7-Day Change Rate*1000',
                            'confirmed_cases_7day': '7-Day Confirmed Cases',
                            'case_avg_7day': '7-Day Case Average',
                            'case_100kres_7day': '7-Day Case Rate Per 100K',
                            'sites_in_zip': 'County Test Sites in Zip',
                            'sites_adj_zips': 'County Test Sites in Adj. Zips',
                           })

df2 = df2.dropna(subset = ['Zip Code'])

df2['Zip Code'] = df2['Zip Code'].astype(int)
df2['Population 2018'] = df2['Population 2018'].astype(int)
df2['Accumulated Confirmed Cases'] = df2['Accumulated Confirmed Cases'].astype(int)
df2['7-Day Confirmed Cases'] = df2['7-Day Confirmed Cases'].astype(int)
df2['County Test Sites in Zip'] = df2['County Test Sites in Zip'].astype(int)
df2['County Test Sites in Adj. Zips'] = df2['County Test Sites in Adj. Zips'].astype(int)

# save df to csv files
df2.to_csv(path_out3, index=False)
df2.to_csv(path_out4, index=False)

df2.head()

,Zip Code,Community,Latitude,Longitude,Population 2018,Date Range,Accumulated Confirmed Cases,Accumulated Rate Per 100K,7-Day Change Rate*1000,7-Day Confirmed Cases,7-Day Case Average,7-Day Case Rate Per 100K,County Test Sites in Zip,County Test Sites in Adj. Zips
0,91901,Alpine,32.805710,-116.695537,17885,1/31/21 - 2/6/21,1131,6323.73,3.98,31,4.43,173.33,0,2
1,91902,Bonita,32.671583,-117.015068,17375,1/31/21 - 2/6/21,1204,6929.50,5.21,43,6.14,247.48,1,4
2,91910,Chula Vista,32.636413,-117.065653,82682,1/31/21 - 2/6/21,8200,9917.52,4.78,269,38.43,325.34,1,5
3,91911,Chula Vista,32.607309,-117.050214,84626,1/31/21 - 2/6/21,10016,11835.61,4.32,298,42.57,352.14,2,1
4,91913,Chula Vista,32.616267,-116.987495,49519,1/31/21 - 2/6/21,4128,8336.19,4.33,123,17.57,248.39,0,4


## Update ArcGIS Online Feature Layer and Maps

In [24]:
# connect to arcgis account
gis = GIS('pro')

In [25]:
# functions
def get_map (map_id):
    '''
    GET MAP DATA FOR SYMBOLOGY CHANGES
    '''  
    m = gis.content.get(map_id)
    data = m.get_data()    
    #Include the below line for prettified JSON
    #print(json.dumps(data, indent=4, sort_keys=True))
    print(m)    
    return data
    
def update_map (map_id, data):
    '''
    UPDATE MAP TO SAVE CHANGES
    '''
    m = gis.content.get(map_id)    
    # Set the item_properties to include the desired update
    properties = {"text": json.dumps(data)}
    # 'Commit' the updates to the Item
    update = m.update(item_properties=properties)
    return update

In [26]:
# feature layer overwrite
#get feature layer containing updated data for maps associated with the COVID-19 dashboards
layer = gis.content.get(layer_id)
layer

layer_collection = FeatureLayerCollection.fromitem(layer)

#call the overwrite() method which can be accessed using the manager property
layer_collection.manager.overwrite(path_out4)

{'success': True}

In [27]:
# get max values to update map symbology - rate100k [14], average [13], changerate [12]
max_rate100k = df2['7-Day Case Rate Per 100K'].max()
max_average = df2['7-Day Case Average'].max()
max_changerate = df2['7-Day Change Rate*1000'].max()

print(max_rate100k, max_average, max_changerate)

494.39 57.14 9.92


In [30]:
# update desktop map
#get map data
data = get_map(map_id)

#adjust symbology for graduated points to reflect new max/min
#MAX rate 100k
data['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_rate100k.item()
data['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_rate100k.item()

#max average
data['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_average.item()
data['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_average.item()

#max change rate
data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_changerate.item()
data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_changerate.item()

#update map to save changes
update = update_map(map_id, data)
update

<Item title:"NIH RADx-UP Weekly COVID-19 Update" type:Web Map owner:jembury8568_SDSUGeo>


True

In [31]:
# update mobile map
#get map data
data2 = get_map(mobilemap_id)

#adjust symbology for graduated points to reflect new max/min
#MAX rate 100k
data2['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_rate100k.item()
data2['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_rate100k.item()

#max average
data2['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_average.item()
data2['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_average.item()

#max change rate
data2['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_changerate.item()
data2['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_changerate.item()

#update map to save changes
update2 = update_map(mobilemap_id, data2)
update2

<Item title:"NIH RADx-UP Weekly COVID-19 Update - Mobile" type:Web Map owner:jembury8568_SDSUGeo>


True

In [32]:
#Open dashboard in browser
webbrowser.open(dash, new=2)

True

## Create Tables for Weekly Report

In [33]:
# Table 1:  Highest 7-Day Case Rates Per 100,000 Resident
df2[['Zip Code', 'Community', '7-Day Case Rate Per 100K', 'County Test Sites in Zip', 'County Test Sites in Adj. Zips']].sort_values(['7-Day Case Rate Per 100K'], ascending=False).head(10)#.to_string(index=False)

,Zip Code,Community,7-Day Case Rate Per 100K,County Test Sites in Zip,County Test Sites in Adj. Zips
46,92083,Vista,494.39,2,2
78,92154,"SD, Otay Mesa",489.93,0,8
60,92114,"SD, Encanto",460.64,1,7
79,92173,San Ysidro,450.60,2,0
8,91935,Jamul,428.44,1,0
47,92084,Vista,427.49,2,3
12,91950,National City,424.87,2,4
59,92113,"SD, Logan Heights",414.21,0,5
27,92027,Escondido,406.66,0,3
25,92025,Escondido,405.75,2,0


In [34]:
# Table 2:  Highest 7-Day Average Case Burdens
df2[['Zip Code', 'Community', '7-Day Case Average', 'County Test Sites in Zip', 'County Test Sites in Adj. Zips']].sort_values(['7-Day Case Average'], ascending=False).head(10)#.to_string(index=False))

,Zip Code,Community,7-Day Case Average,County Test Sites in Zip,County Test Sites in Adj. Zips
78,92154,"SD, Otay Mesa",57.14,0,8
3,91911,Chula Vista,42.57,2,1
60,92114,"SD, Encanto",38.71,1,7
2,91910,Chula Vista,38.43,1,5
12,91950,National City,35.57,2,4
52,92105,"SD, City Heights",32.43,1,3
27,92027,Escondido,31.14,0,3
22,92020,El Cajon,30.29,1,1
47,92084,Vista,30.14,2,3
25,92025,Escondido,29.86,2,0


In [35]:
# Table 3:  Highest 7-Day Average Change Rate
df2[['Zip Code', 'Community', '7-Day Change Rate*1000', 'County Test Sites in Zip', 'County Test Sites in Adj. Zips']].sort_values(['7-Day Change Rate*1000'], ascending=False).head(10)#.to_string(index=False))

,Zip Code,Community,7-Day Change Rate*1000,County Test Sites in Zip,County Test Sites in Adj. Zips
57,92110,"SD, Old Town",9.92,1,1
8,91935,Jamul,9.27,1,0
19,92011,Carlsbad,9.26,0,0
32,92054,Oceanside,7.76,1,3
41,92071,Santee,7.70,1,1
16,92008,Carlsbad,7.61,0,1
39,92067,Rancho Santa Fe,7.60,0,1
46,92083,Vista,7.20,2,2
47,92084,Vista,7.00,2,3
31,92040,Lakeside,6.75,0,2
